# Taller 4

## Carga CSV a MySQL con carga inicial e incremental

*mysql -u root -p*


In [3]:
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os

load_dotenv()
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_NAME = os.getenv('DB_NAME')

# Agregar parámetros de conexión para manejar la autenticación MySQL 8.0+
DB_URL = f'mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}?charset=utf8mb4&auth_plugin_map=caching_sha2_password:mysql_native_password'

# Crear engine con configuraciones adicionales
engine = create_engine(
    DB_URL,
    pool_pre_ping=True,
    pool_recycle=300,
    echo=False
)

ddl = """
    CREATE TABLE IF NOT EXISTS laboratorio (
    laboratorio_id INT AUTO_INCREMENT PRIMARY KEY,
    cliente VARCHAR(100),
    producto VARCHAR(100),
    categoria VARCHAR(100),
    precio_unitario DECIMAL(10, 2),
    unidades INT,
    fecha DATE
    );
"""

try:
    with engine.connect() as connection:
        connection.execute(text(ddl))
        connection.commit()
    print("Tabla creada exitosamente.")
except Exception as e:
    print(f"Error al conectar con la base de datos: {e}")
    print("Intentando conexión alternativa...")
    
    # Intentar con configuración alternativa
    DB_URL_ALT = f'mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}?charset=utf8mb4'
    engine_alt = create_engine(DB_URL_ALT, pool_pre_ping=True)
    
    try:
        with engine_alt.connect() as connection:
            connection.execute(text(ddl))
            connection.commit()
        print("Tabla creada exitosamente con configuración alternativa.")
        engine = engine_alt  # Usar la configuración alternativa
    except Exception as e2:
        print(f"Error con configuración alternativa: {e2}")
        print("Verificar que MySQL esté ejecutándose: docker-compose up -d")

Tabla creada exitosamente.
